The three JSON files from data scrape.ipynb are imported to create the cumulative and the mobile JSON files

In [1]:
import pandas as pd
import json
from json import loads, dumps

df_desktop = pd.read_json('inter_desktop_start201507-end202310.json')['articles']
df_desktop = pd.json_normalize(df_desktop)

df_mobile_app = pd.read_json('inter_mobile_app_start201507-end202310.json')['articles']
df_mobile_app = pd.json_normalize(df_mobile_app)

df_mobile_web = pd.read_json('inter_mobile_web_start201507-end202310.json')['articles']
df_mobile_web = pd.json_normalize(df_mobile_web)

From the below, we have the basic information of the number of views from different platforms.

In [2]:
df_desktop.describe()

,views
count,1.288600e+05
mean,1.144953e+04
std,3.720301e+04
min,0.000000e+00
25%,5.370000e+02
50%,2.787000e+03
75%,1.188100e+04
max,3.355060e+06


In [3]:
df_mobile_app.describe()

,views
count,128860.000000
mean,875.145887
std,4109.631416
min,0.000000
25%,16.000000
50%,144.000000
75%,824.000000
max,516287.000000


In [4]:
df_mobile_web.describe()

,views
count,1.288600e+05
mean,1.822776e+04
std,7.476695e+04
min,0.000000e+00
25%,4.120000e+02
50%,3.001500e+03
75%,1.707425e+04
max,7.736645e+06


In [5]:
df_mobile_web.columns

Index(['project', 'article', 'granularity', 'timestamp', 'access', 'agent',
       'views'],
      dtype='object')

In [6]:
df_mobile_web.head()

,project,article,granularity,timestamp,access,agent,views
0,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,2020010100,mobile-web,user,2241
1,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,2020020100,mobile-web,user,4955
2,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,2020030100,mobile-web,user,4427
3,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,2020040100,mobile-web,user,9540
4,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,2020050100,mobile-web,user,7878


In [7]:
# remove extra bits from timestamp by extracting only year and month out form timestamp
# concat year and month together for future group by

df_desktop['month'] = df_desktop['timestamp'].str.slice(start=4,stop=6)
df_desktop['year'] =df_desktop['timestamp'].str.slice(stop=4)
df_desktop['time'] = df_desktop[['year', 'month']].agg('-'.join, axis=1)
df_desktop = df_desktop.drop(columns=['year', 'month', 'timestamp'])
df_desktop

,project,article,granularity,access,agent,views,time
0,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,desktop,user,1209,2020-01
1,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,desktop,user,2944,2020-02
2,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,desktop,user,2612,2020-03
3,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,desktop,user,4530,2020-04
4,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,desktop,user,3952,2020-05
...,...,...,...,...,...,...,...
128855,en.wikipedia,Zorba_the_Greek_(film),monthly,desktop,user,4398,2023-05
128856,en.wikipedia,Zorba_the_Greek_(film),monthly,desktop,user,3865,2023-06
128857,en.wikipedia,Zorba_the_Greek_(film),monthly,desktop,user,5919,2023-07
128858,en.wikipedia,Zorba_the_Greek_(film),monthly,desktop,user,5789,2023-08


### For the JSON file that's specifically for mobile platform, we join on article and timestamp to prepare for the summation of total view from both mobile-web and -app. We also further join on project, granularity and agent to remove redundancy before-hand

In [8]:
# join on article and timestamp to prepare for the summation of total view from both mobile-web and -app
# further join on project, granularity and agent to remove redundancy before hand

mobile = pd.merge(df_mobile_app, df_mobile_web, on=['project','article','timestamp','granularity','agent'])
mobile

,project,article,granularity,timestamp,access_x,agent,views_x,access_y,views_y
0,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,2020010100,mobile-app,user,65,mobile-web,2241
1,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,2020020100,mobile-app,user,152,mobile-web,4955
2,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,2020030100,mobile-app,user,120,mobile-web,4427
3,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,2020040100,mobile-app,user,284,mobile-web,9540
4,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,2020050100,mobile-app,user,231,mobile-web,7878
...,...,...,...,...,...,...,...,...,...
128855,en.wikipedia,Zorba_the_Greek_(film),monthly,2023050100,mobile-app,user,493,mobile-web,8134
128856,en.wikipedia,Zorba_the_Greek_(film),monthly,2023060100,mobile-app,user,484,mobile-web,8282
128857,en.wikipedia,Zorba_the_Greek_(film),monthly,2023070100,mobile-app,user,765,mobile-web,14055
128858,en.wikipedia,Zorba_the_Greek_(film),monthly,2023080100,mobile-app,user,1203,mobile-web,12530


In [9]:
mobile['views'] = mobile['views_x']+mobile['views_y']

In [10]:
# take out the access type for mobile

mobile['access'] = mobile['access_x'].str.slice(stop=6)

In [11]:
# remove extra bits from timestamp by extracting only year and month out form timestamp
# concat year and month together for future group by

mobile['month'] = mobile['timestamp'].str.slice(start=4,stop=6)
mobile['year'] =mobile['timestamp'].str.slice(stop=4)
mobile['time'] = mobile[['year', 'month']].agg('-'.join, axis=1)

In [12]:
# remove the redundant columns resulting from joining after the computation

mobile = mobile.drop(columns=['access_x', 'access_y', 'views_x', 'views_y', 'year', 'month', 'timestamp'])
mobile

,project,article,granularity,agent,views,access,time
0,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,user,2306,mobile,2020-01
1,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,user,5107,mobile,2020-02
2,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,user,4547,mobile,2020-03
3,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,user,9824,mobile,2020-04
4,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,user,8109,mobile,2020-05
...,...,...,...,...,...,...,...
128855,en.wikipedia,Zorba_the_Greek_(film),monthly,user,8627,mobile,2023-05
128856,en.wikipedia,Zorba_the_Greek_(film),monthly,user,8766,mobile,2023-06
128857,en.wikipedia,Zorba_the_Greek_(film),monthly,user,14820,mobile,2023-07
128858,en.wikipedia,Zorba_the_Greek_(film),monthly,user,13733,mobile,2023-08


### For the JSON file that's for all platforms, we join on article and timestamp to prepare for the summation of total view from both desktop and the resulting DataFrame that's solely about mobile from the above. We also further join on project, granularity and agent to remove redundancy before-hand

In [13]:
# join on article and timestamp to prepare for the summation of total view from both mobile and desktop
# further join on project, granularity and agent to remove redundancy before hand

cumulative = pd.merge(mobile, df_desktop, on=['project','article','granularity','agent','time'])
cumulative

,project,article,granularity,agent,views_x,access_x,time,access_y,views_y
0,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,user,2306,mobile,2020-01,desktop,1209
1,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,user,5107,mobile,2020-02,desktop,2944
2,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,user,4547,mobile,2020-03,desktop,2612
3,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,user,9824,mobile,2020-04,desktop,4530
4,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,user,8109,mobile,2020-05,desktop,3952
...,...,...,...,...,...,...,...,...,...
128855,en.wikipedia,Zorba_the_Greek_(film),monthly,user,8627,mobile,2023-05,desktop,4398
128856,en.wikipedia,Zorba_the_Greek_(film),monthly,user,8766,mobile,2023-06,desktop,3865
128857,en.wikipedia,Zorba_the_Greek_(film),monthly,user,14820,mobile,2023-07,desktop,5919
128858,en.wikipedia,Zorba_the_Greek_(film),monthly,user,13733,mobile,2023-08,desktop,5789


In [14]:
cumulative['views'] = cumulative['views_x']+cumulative['views_y']

In [15]:
# concat both accesses for the cumulative JSON file

cumulative['access'] = cumulative[['access_x', 'access_y']].agg('/'.join, axis=1)

In [16]:
# # remove extra bits from timestamp by extracting only year and month out form timestamp
# # concat year and month together for future group by

# cumulative['month'] = cumulative['timestamp'].str.slice(start=4,stop=6)
# cumulative['year'] =cumulative['timestamp'].str.slice(stop=4)
# cumulative['time'] = cumulative[['year', 'month']].agg('-'.join, axis=1)

In [17]:
# remove the redundant columns resulting from joining after the computation

cumulative = cumulative.drop(columns=['access_x', 'access_y', 'views_x', 'views_y'])
cumulative

,project,article,granularity,agent,time,views,access
0,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,user,2020-01,3515,mobile/desktop
1,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,user,2020-02,8051,mobile/desktop
2,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,user,2020-03,7159,mobile/desktop
3,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,user,2020-04,14354,mobile/desktop
4,en.wikipedia,Everything_Everywhere_All_at_Once,monthly,user,2020-05,12061,mobile/desktop
...,...,...,...,...,...,...,...
128855,en.wikipedia,Zorba_the_Greek_(film),monthly,user,2023-05,13025,mobile/desktop
128856,en.wikipedia,Zorba_the_Greek_(film),monthly,user,2023-06,12631,mobile/desktop
128857,en.wikipedia,Zorba_the_Greek_(film),monthly,user,2023-07,20739,mobile/desktop
128858,en.wikipedia,Zorba_the_Greek_(film),monthly,user,2023-08,19522,mobile/desktop


In [21]:
# produce the combination of mobile data from mobile-app and -web

out_mobile = mobile.to_json(orient="records")
parsed_mobile = loads(out_mobile)

with open("academy_monthly_mobile_start201507-end202310.json", "a") as outfile:
    outfile.write(json.dumps(parsed_mobile,indent=4))

In [22]:
# produce the combination of data from the resulting mobile DataFrame obtained from the previous codes 
# and the Dataframe of the desktop access

out_cumulative = cumulative.to_json(orient="records")
parsed_cumulative = loads(out_cumulative)

with open("academy_monthly_cumulative_start201507-end202310.json", "a") as outfile:
    outfile.write(json.dumps(parsed_cumulative,indent=4))

In [23]:
# produce the combination of data from the resulting mobile DataFrame obtained from the previous codes 
# and the Dataframe of the desktop access

out_desktop = df_desktop.to_json(orient="records")
parsed_desktop = loads(out_desktop)

with open("academy_monthly_desktop_start201507-end202310.json", "a") as outfile:
    outfile.write(json.dumps(parsed_desktop,indent=4))